In [40]:
# Create Naive Bayes classifier using TD-IDF vectorizer and pickle results. This model can be 
# trained on job resumes or Indeed job postings. Steps include:
# 1. Set the model parameters
# 2. Getting a combined list of salaries and only using job titles with 500+ salary records
# 3. Getting a list of resumes using this list of job titles and remove any job titles with 
#    less than 500 resumes
# 4. Run TD-IDF vectorizer and Naive Bayes model training
# 5. Test the model using "List Most Relevant Skills"
# 6. Test the model using "Document Similarity Score"

In [1]:
import datetime
import math
import numpy as np
import pandas as pd
import pickle
from itertools import product
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import FunctionTransformer
from imblearn.pipeline import Pipeline 


# Custom function in functions folder
from functions.word_preprocessing import *

pd.set_option('display.max_colwidth', 500)

/home/kwheatley/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'
directory = '/mnt/disks/mnt_dir/data/'

# Model Parameters

In [3]:
# # Best parameters for doc similarity
# parameters = {
#                 "doc_type":"indeed_resume", # Use "indeed_resume" or "indeed_postings"
#                 "min_salary_records":100, # Filter out all jobs with less than specified salary records
#                 "min_job_summaries":1000, # Filter out all jobs with less than specified job summaries
#                 "min_ngram":1, # For TD-IDF vectorizer
#                 "max_ngram":3, # For TD-IDF vectorizer
#                 "min_df":5, # For TD-IDF vectorizer, ignore features in less than this number of documents
#                 "train_test_split":0.05, # For train-test split
#                 "random_state":1, # For train-test split
#                 "alpha":0.02, # For Naive Bayes model 0.02 is current best
#                 "num_skills":50, # Number of skill to show per job 
#                 "max_number_records":2500, # Number of records in each class, SMOTE is used to fill small classes
# }

In [4]:
# Best parameters for skills

parameters = {
                "doc_type":"indeed_resume", # Use "indeed_resume" or "indeed_postings"
                "min_salary_records":100, # Filter out all jobs with less than specified salary records
                "min_job_summaries":1000, # Filter out all jobs with less than specified job summaries
                "min_ngram":3, # For TD-IDF vectorizer
                "max_ngram":3, # For TD-IDF vectorizer
                "min_df":5, # For TD-IDF vectorizer, ignore features in less than this number of documents
                "train_test_split":0.05, # For train-test split
                "random_state":1, # For train-test split
                "alpha":0.02, # For Naive Bayes model 0.02 is current best
                "num_skills":100, # Number of skill to show per job 
                "max_number_records":5000, # Number of records in each class, SMOTE is used to fill small classes
}

# Load Salary Datasets

In [46]:
# Load the both salary datasets
salary1 = pd.read_csv(directory+'02_salaries_h1b.csv')
salary2 = pd.read_csv(directory+'02_salaries_greencard.csv')

# Combine salary datasets
temp_salary1 = salary1[['role','city','state','start_year','cleaned_job_title','experiences','salary']]
temp_salary2 = salary2[['job_title','city','state','decision_year','cleaned_job_title','experiences','salary_amount']]
temp_salary1.columns = ['original_role','city','state','start_year','cleaned_job_title','experiences','salary']
temp_salary2.columns = ['original_role','city','state','start_year','cleaned_job_title','experiences','salary']
combined_salaries = temp_salary1.append(temp_salary2)

# Remove salaries with null value and convert to int
combined_salaries = combined_salaries[~combined_salaries.salary.isnull()]
combined_salaries.salary = combined_salaries.salary.astype(int)

# Fill any NaN fields with no_value and convert each column into a list
combined_salaries = combined_salaries.fillna('no_value')
combined_salaries.experiences = combined_salaries.experiences.apply(lambda x: 
                                                list([item.strip() for item in x.split(',')]))
combined_salaries.original_role = combined_salaries.original_role.apply(lambda x: [x])
combined_salaries.city = combined_salaries.city.apply(lambda x: [x])
combined_salaries.state = combined_salaries.state.apply(lambda x: [x])
combined_salaries.start_year = combined_salaries.start_year.apply(lambda x: [x])
combined_salaries.cleaned_job_title = combined_salaries.cleaned_job_title.apply(lambda x: [x])
combined_salaries.salary = combined_salaries.salary.apply(lambda x: [x])

# Perform a pivot on the columns to split out rows with multiple experience level qualifiers
combined_salaries = pd.DataFrame([j for i in combined_salaries.values for j in product(*i)],
                                      columns = combined_salaries.columns)

# Get List of Jobs

In [47]:
# Choose all jobs with `min_salary_records` or more records
temp = combined_salaries.groupby('cleaned_job_title').count().salary.reset_index()
jobs_to_model = temp[temp.salary >= parameters['min_salary_records']]
pd.DataFrame(jobs_to_model.cleaned_job_title.unique()).to_csv(directory+'03_relevant_job_titles.csv',index=False)
combined_salaries = combined_salaries[combined_salaries.cleaned_job_title\
                                                       .fillna('').isin(jobs_to_model.cleaned_job_title)]
print("Number of jobs with "+str(parameters['min_salary_records'])+"+ salary records:", jobs_to_model.cleaned_job_title.count())

# combined_salaries.groupby(['cleaned_job_title'])\
# .salary.agg(['count','mean','min','max','median','std']).to_csv("sample.csv")

Number of jobs with 100+ salary records: 867


# Load Job Summaries

In [48]:
# Load resume data
if parameters['doc_type'] == 'indeed_resume':
    data = pd.read_csv(directory+'02_resumes_work.csv')
if parameters['doc_type'] == 'indeed_postings':
    data = pd.read_csv(directory+'02_job_posts_indeed.csv')

# Remove all null cleaned_job_title records
jobs_descriptions = data[~data.cleaned_job_title.isnull()]

In [49]:
# Filter to only jobs specified by the jobs_to_model list
jobs_descriptions = jobs_descriptions[jobs_descriptions.cleaned_job_title\
                                      .isin(jobs_to_model.cleaned_job_title)]

# Rename job summary field
if parameters['doc_type'] == 'indeed_resume':
    jobs_descriptions.rename(columns = {'descript':'summary_text'}, inplace=True)

In [50]:
# Print the number of records left after removing irrelevant jobs
print("Number of resume entries available:", 
      str(jobs_descriptions.cleaned_job_title.count()) +"/"+
      str(data.cleaned_job_title.count()))

Number of resume entries available: 997695/3200066


In [51]:
# Remove all jobs without `min_job_summaries` or more resume entries
cnt_resumes_available = jobs_descriptions.groupby('cleaned_job_title')\
                                .count().reset_index()
cnt_resumes_available = list(cnt_resumes_available[
            cnt_resumes_available.summary_text>parameters['min_job_summaries']].cleaned_job_title)
jobs_descriptions = jobs_descriptions[jobs_descriptions.cleaned_job_title\
                       .isin(cnt_resumes_available)]

# Save off list of resume ids
if parameters['doc_type'] == 'indeed_resume':
    # Save the list of resume ids for resumes being used
    pd.DataFrame(jobs_descriptions.resume_id.unique())\
                .to_csv(directory+'03_relevant_resume_ids.csv',index=False)
        
# Remove duplicate data
jobs_descriptions = jobs_descriptions[['cleaned_job_title','summary_text','from_year']].drop_duplicates()

print("Number of jobs with "+str(parameters['min_job_summaries'])+"+ resume entries:", len(cnt_resumes_available))
print("Number of resume entries available now:", jobs_descriptions.cleaned_job_title.count())

Number of jobs with 1000+ resume entries: 152
Number of resume entries available now: 849706


# Save Salary Data for App

In [52]:
# This code saves the cleaned salary information back to the main data folder
combined_salaries.to_csv(directory+'03_cleaned_salaries_for_app.csv',index=False)

# Data Preprocess

In [53]:
# This code samples the number of records to remove excessive numbers
new_job_descriptions = pd.DataFrame()
for name, group in jobs_descriptions.groupby('cleaned_job_title'):
    if group[group.from_year == 2016].cleaned_job_title.count() >= parameters['max_number_records']:
        new_job_descriptions = pd.concat([new_job_descriptions,group[group.from_year < 2017]\
            .sort_values(by='from_year', ascending=False).head(parameters['max_number_records'])])
    elif group[(group.from_year == 2016) | (group.from_year == 2017)].cleaned_job_title.count() >= parameters['max_number_records']:
        new_job_descriptions = pd.concat([new_job_descriptions,group[group.from_year < 2018]\
            .sort_values(by='from_year', ascending=False).head(parameters['max_number_records'])])
    else:
        new_job_descriptions = pd.concat([new_job_descriptions,group\
            .sort_values(by='from_year', ascending=False).head(parameters['max_number_records'])])

new_job_descriptions.count()

cleaned_job_title    483903
summary_text         483903
from_year            483903
dtype: int64

In [54]:
# jobs_descriptions.groupby(['cleaned_job_title']).summary_text.count().sort_values(ascending=False).head(70)

In [55]:
x_data = new_job_descriptions.summary_text
y_labels = new_job_descriptions.cleaned_job_title

In [6]:
# Split the data into test and train datasets
X_train, X_test, y_train, y_test = train_test_split(x_data, 
                                                    y_labels,
                                                    test_size=parameters['train_test_split'],
                                                    random_state=parameters['random_state'])

# Setup Pipeline

In [57]:
preprocess = FunctionTransformer(preprocess_list)
smt = SMOTE()
tfidf = TfidfVectorizer(min_df=parameters['min_df'], ngram_range=(parameters['min_ngram'], parameters['max_ngram']))
nb = MultinomialNB(alpha=parameters['alpha'])
preprocess = FunctionTransformer(preprocess_list, validate=False)

pipeline = Pipeline([('preprocess', preprocess), ('tfidf', tfidf), ('smt', smt), ('nb', nb)])

# Train Model

In [58]:
print('X_train:', len(X_train))
print('y_train:', len(y_train))

print("Start:", datetime.datetime.now())
pipeline.fit(X_train,y_train)
print("End:", datetime.datetime.now())

X_train: 459707
y_train: 459707
Start: 2018-07-24 23:43:16.778521
End: 2018-07-25 00:07:21.414853


In [ ]:
y_pred = pipeline.predict(X_test)
print('Accuracy: %.2f%%' % (accuracy_score(y_test, y_pred) * 100))

# Accuracy: 44.81% for doc similarity
# Accuracy: 35.53% for skills

Accuracy: 35.53%


In [60]:
print('f1_score: ', f1_score(y_test, y_pred, average="macro"))
print('precision_score: ', precision_score(y_test, y_pred, average="macro"))
print('recall_score: ', recall_score(y_test, y_pred, average="macro"))   

f1_score:  0.323935852893
precision_score:  0.336582884615
recall_score:  0.332000061005


In [61]:
precision, recall, fscore, support = score(y_test, y_pred)

metrics = pd.DataFrame(list(zip(pipeline.classes_, precision, recall, fscore, support)))
metrics.columns = ['class','precision', 'recall', 'fscore', 'support']
metrics.sort_values(by='fscore',ascending=True).head()

,class,precision,recall,fscore,support
129,specialist,0.071429,0.030769,0.043011,65
87,operations analyst,0.078947,0.040541,0.053571,74
31,ceo,0.085714,0.044118,0.058252,68
23,business development,0.067416,0.073171,0.070175,82
12,assistant,0.190476,0.054422,0.084656,147


In [62]:
# import collections
# counted = collections.Counter(y_test)

# from operator import itemgetter
# import heapq
# import collections
# def least_common_values(array, to_find=None):
#     counter = collections.Counter(array)
#     if to_find is None:
#         return sorted(counter.items(), key=itemgetter(1), reverse=False)
#     return heapq.nsmallest(to_find, counter.items(), key=itemgetter(1))

# # counted.most_common()
# least_common_values(counted, 50)

In [63]:
# From the test population, get the most confused labels

# pd.options.display.max_rows = 10000
predictions = pd.DataFrame(list(zip(y_test, y_pred)))
predictions.columns=['actual','prediction']
predictions['count']=1
pred_group = predictions.groupby(['actual','prediction']).count().reset_index()
pred_group[(pred_group.actual != pred_group.prediction) 
           & (pred_group.prediction!='account executive')
          ].sort_values(by='count',ascending=False).head(20)
# .to_csv('most_confusion.csv')

,actual,prediction,count
117,accountant,staff accountant,61
2369,human resources specialist,human resources manager,61
4811,staff accountant,accountant,53
3225,process engineer,manufacturing engineer,51
4946,systems administrator,network administrator,48
2620,j2ee engineer,java software engineer,47
2636,java software engineer,j2ee engineer,42
3032,network administrator,systems administrator,42
1304,controller,accounting manager,40
5096,teacher,preschool teacher,38


# List Most Relevant Skills

In [ ]:
# This code finds the top parameters['num_skills'] of features to show the user. It filters out any 
# ngram where the same n-1 version of the ngram is shown. This cuts down on repetition.

label_id = 93

print(pipeline.classes_[label_id])
print('-------')

features_list = []
topn_class1 = sorted(zip(pipeline.named_steps['nb'].coef_[label_id], 
                         pipeline.named_steps['tfidf'].get_feature_names()))[-parameters['num_skills']:]
for coef, feat in topn_class1:
    features_list.append(feat)

accepted_skill_list = [pipeline.classes_[label_id]]
for potential_skill in sorted(features_list, key=lambda x: -len(x.split())):
    highest_match = len(potential_skill.split())
    for accepted_skill in accepted_skill_list:
        leftovers = list(set(potential_skill.split()) - set(accepted_skill.split()))
        if len(leftovers) < highest_match:
            highest_match = len(leftovers)
    if highest_match > 1:
        accepted_skill_list.append(potential_skill)
accepted_skill_list = accepted_skill_list[1:]
shuffle(accepted_skill_list)

for skill in accepted_skill_list:
    print(skill)

product manager
-------


# Document Similarity Score

In [65]:
# This code returns the prediction probabilities for an example input

example_index = 40
print(y_test[example_index:example_index+1])

print()
print("---------------------")
example = X_test[example_index:example_index+1]
print(example)

print()
print("---------------------")

job_rankings = list(zip(pipeline.predict_proba(example)[0],pipeline.classes_))
sorted(job_rankings,reverse=True)[:20]

299648    consultant
Name: cleaned_job_title, dtype: object

---------------------
299648    .as part of academically sponsored team-based project .• Conducted industry research and competitive benchmarking in order to develop pricing strategy .• Determined feasibility of program based on financial analysis of Bay Cove Academy.
Name: summary_text, dtype: object

---------------------


[(0.22726698136641141, 'financial analyst'),
 (0.10846793866030462, 'marketing specialist'),
 (0.090911912318914545, 'chief financial officer'),
 (0.076218383979244672, 'business development manager'),
 (0.047572097864376342, 'analyst'),
 (0.037165532420553309, 'technical writer'),
 (0.033379704417007695, 'consultant'),
 (0.024299218120848249, 'business manager'),
 (0.021285235634691892, 'product marketing manager'),
 (0.018763913424894845, 'program manager'),
 (0.014960565077392386, 'accounting'),
 (0.013389055843904769, 'marketing manager'),
 (0.013364387274254676, 'product manager'),
 (0.009982822959913918, 'business development'),
 (0.0098005846809103812, 'business consultant'),
 (0.0083345510721833452, 'sales manager'),
 (0.0077160757680939246, 'operations manager'),
 (0.007528294906168775, 'buyer'),
 (0.0072209841150210619, 'operations analyst'),
 (0.0067130493753592216, 'creative director')]

# Save New Model

In [66]:
# This code saves the model to the models folder
directory = '/mnt/disks/mnt_dir/'

save_time = re.sub('[^A-Za-z0-9]+', '', str(datetime.datetime.now()))
print(save_time)

write_param = open(directory+"models/" + save_time + '_parameters.txt','w')
for key in parameters:
    write_param.write(key + "=" + str(parameters[key]) + '\n')
write_param.close()

# Save preprocessed x data
pickling_on = open(directory+"models/"+save_time+"_x_data.pkl","wb")
pickle.dump(x_data, pickling_on)
pickling_on.close()

# Save preprocessed y labels
pickling_on = open(directory+"models/"+save_time+"_y_labels.pkl","wb")
pickle.dump(y_labels, pickling_on)
pickling_on.close()

# Save NB model
pickling_on = open(directory+"models/"+save_time+"_pipeline_model.pkl","wb")
pickle.dump(pipeline, pickling_on)
pickling_on.close()

20180725002822539809


# Load Model

In [5]:
# This code loads an old model
directory = '/mnt/disks/mnt_dir/'

# save_time = '20180724220628349336' # Currently best model for doc similarity
save_time = '20180725002822539809' # Currently best model for skills

pickling_on = open(directory+"models/"+save_time+"_x_data.pkl","rb")
x_data = pickle.load(pickling_on)
pickling_on.close()

pickling_on = open(directory+"models/"+save_time+"_y_labels.pkl","rb")
y_labels = pickle.load(pickling_on)
pickling_on.close()

pickling_on = open(directory+"models/"+save_time+"_pipeline_model.pkl","rb")
pipeline = pickle.load(pickling_on)
pickling_on.close()

# End